# 6.5 循环神经网络的简洁实现

本节将使用PyTorch来更简洁地实现基于循环神经网络的语言模型。首先，我们读取周杰伦专辑歌词数据集。

In [1]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

## 6.5.1 定义模型

PyTorch中的`nn`模块提供了循环神经网络的实现。下面构造一个含单隐藏层、隐藏单元个数为256的循环神经网络层`rnn_layer`。

In [4]:
num_hiddens = 256

# rnn_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens) # 已测试
rnn_layer = nn.RNN(input_size=vocab_size, 
                   hidden_size=num_hiddens
                  )

与上一节中实现的循环神经网络不同，这里`rnn_layer`的输入形状为(时间步数, 批量大小, 输入个数)。其中输入个数即one-hot向量长度（词典大小）。此外，`rnn_layer`作为`nn.RNN`实例，在前向计算后会分别返回输出和隐藏状态h，其中输出指的是隐藏层在**各个时间步**上计算并输出的隐藏状态，它们通常作为后续输出层的输入。需要强调的是，该“输出”本身并不涉及输出层计算，形状为(时间步数, 批量大小, 隐藏单元个数)。而`nn.RNN`实例在前向计算返回的隐藏状态指的是隐藏层在**最后时间步**的隐藏状态：当隐藏层有多层时，每一层的隐藏状态都会记录在该变量中；对于像长短期记忆（LSTM），隐藏状态是一个元组(h, c)，即hidden state和cell state。我们会在本章的后面介绍长短期记忆和深度循环神经网络。关于循环神经网络（以LSTM为例）的输出，可以参考下图（[图片来源](https://stackoverflow.com/questions/48302810/whats-the-difference-between-hidden-and-output-in-pytorch-lstm/48305882)）。

来看看我们的例子，输出形状为(时间步数, 批量大小, 隐藏单元个数)，隐藏状态h的形状为(层数, 批量大小, 隐藏单元个数)。

In [5]:
num_steps = 35
batch_size = 2
state = None


X = torch.rand(num_steps, batch_size, vocab_size)
Y, state_new = rnn_layer(X, state)


print(Y.shape, len(state_new), state_new[0].shape)

torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


> 如果`rnn_layer`是`nn.LSTM`实例，那么上面的输出是什么？

接下来我们继承`Module`类来定义一个完整的循环神经网络。它首先将输入数据使用one-hot向量表示后输入到`rnn_layer`中，然后使用全连接输出层得到输出。输出个数等于词典大小`vocab_size`。

In [6]:
# 本类已保存在d2lzh_pytorch包中方便以后使用
class RNNModel(nn.Module):
    
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1) 
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)
        self.state = None

    def forward(self, inputs, state): # inputs: (batch, seq_len)
        
        # 获取one-hot向量表示
        X = d2l.to_onehot(inputs, self.vocab_size) # X是个list
        Y, self.state = self.rnn(torch.stack(X), state)
        
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.dense(Y.view(-1, Y.shape[-1]))
        return output, self.state

## 6.5.2 训练模型

同上一节一样，下面定义一个预测函数。这里的实现区别在于前向计算和初始化隐藏状态的函数接口。

In [7]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    
    state = None
    output = [char_to_idx[prefix[0]]] # output会记录prefix加上输出
    
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        
        if state is not None:
            if isinstance(state, tuple): # LSTM, state:(h, c)  
                state = (state[0].to(device), state[1].to(device))
            else:   
                state = state.to(device)

        (Y, state) = model(X, state)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))

    return ''.join([idx_to_char[i] for i in output])

让我们使用权重为随机值的模型来预测一次。

In [8]:
model = RNNModel(rnn_layer, vocab_size).to(device)

# 预测
predict_rnn_pytorch('分开', 10, model, vocab_size, 
                    device, 
                    idx_to_char, 
                    char_to_idx
                   )

'分开只暖只久只深直只啸只'

接下来实现训练函数。算法同上一节的一样，但这里只使用了相邻采样来读取数据。

In [9]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    state = None
    
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        
        # 相邻采样
        data_iter = d2l.data_iter_consecutive(corpus_indices, 
                                              batch_size, 
                                              num_steps, 
                                              device) 
        
        for X, Y in data_iter:
            if state is not None:

                # 使用detach函数从计算图分离隐藏状态, 这是为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state = (state[0].detach(), state[1].detach())
                else:   
                    state = state.detach()

            # output: 形状为(num_steps * batch_size, vocab_size)
            (output, state) = model(X, state) 
            
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            
            # 梯度裁剪
            d2l.grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        
        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
            
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

使用和上一节实验中一样的超参数（除了学习率）来训练模型。

In [11]:

# 注意这里的学习率设置
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2 

pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']


# 训练
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 3.732021, time 0.01 sec
 - 分开 我想 你爱你 我想你你想很久 别要你 说过了口让我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的
 - 不分开不 我想想你你你我 我想想你你你我 我想想你你你我 我想想你你你我 我想想你你你我 我想想你你你我 
epoch 100, perplexity 1.173272, time 0.01 sec
 - 分开不会像语样没 你在没有多 就是我想想你想要 你的爸你 爸 就怎么 你手不会我大你 不要再这样打我妈妈
 - 不分开不了我 想这样种担走 我想到 你你在每天妈想就想想要说 三对不起 他知后觉 我该好好生活 我该好好生
epoch 150, perplexity 1.054971, time 0.01 sec
 - 分开不会像语样没 你说我爱你 这样打我妈妈是我以想你 爱女人 漂亮的让我面红的可爱女人 温柔的让我心疼的
 - 不分开不知我爱妈你已经我的我都想要你的过我 让我想多这样 我有多难熬  穿你 一个人口妈  只那纵 让静为
epoch 200, perplexity 1.032642, time 0.01 sec
 - 分开不会像错样没 你到 爱我的爸爸一直打我妈妈 就远方喝醉温休  为什么 你说 我已经很久 别想我 说散
 - 不分开不知我觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该
epoch 250, perplexity 1.019459, time 0.01 sec
 - 分开 我想大声宣布 对你依依不舍 连隔壁邻居都猜到我现在的感受 河边的风 在吹着头发飘动 牵着你的手 一
 - 不分开不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该


## 小结

* PyTorch的`nn`模块提供了循环神经网络层的实现。
* PyTorch的`nn.RNN`实例在前向计算后会分别返回输出和隐藏状态。该前向计算并不涉及输出层计算。